In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis , LocalOutlierFactor
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore")


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](http://)<font color = 'purple'>
    
# Introduction
<font color = 'black'>

We are gonna use KNN for this dataset.


<font color = 'purple'>

Content:
1. [Load and Check Data](#1) 
1. [Exploratory Data Analysis](#2)
1. [Outlier Detection](#3)
1. [Train-Test Split](#4)
1. [KNN](#5) 
1. [NCA](#6) 

<a id="1"></a>
<font color='purple'>
## Load and Check Data

In [ ]:
data=pd.read_csv("/kaggle/input/breast-cancer-wisconsin-data/data.csv")

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.columns

Id and Unnamed:32 columns are unnecessary for machine learning so we are gonna drop them.

In [ ]:
data.drop(['id','Unnamed: 32'], inplace=True ,axis=1)

I decided to change name of the column diagnosis with target.

In [ ]:
data.rename(columns= {"diagnosis":"target"}, inplace=True)

In [ ]:
sns.countplot(data["target"])
plt.show()

In [ ]:
data.target.value_counts()

Changing M to 1 and B to 0 for easier machine learning process.

In [ ]:
data["target"]=[1 if i.strip()== "M" else 0 for i in data["target"]]

In [ ]:
data.target.unique()

In [ ]:
data.shape

<a id="2"></a>
<font color='purple'>
## Exporatory Data Analysis

Correlation:

In [ ]:
corr_matrix= data.corr()
sns.clustermap(corr_matrix, annot=True , fmt=".2f")
plt.title("Correlation Between Features")
plt.show()

It is too complicated like that. So i am gonna check the ones with correlation bigger than 0.75

In [ ]:
threshold=0.75
filtre=np.abs(corr_matrix["target"]> threshold)
corr_features=corr_matrix.columns[filtre].tolist()
sns.clustermap(data[corr_features].corr(), annot=True, fmt=".2f")
plt.title("Correlation Between Features w Corr Threshold 0.75")
plt.show()

Box Plot:

In [ ]:
data_melted=pd.melt(data, id_vars="target", var_name="features", value_name="value")
plt.figure(figsize=(10,8))
sns.boxplot(x="features", y="value", hue="target", data=data_melted)
plt.xticks(rotation=90)
plt.show()

Pairplot:

In [ ]:
sns.pairplot(data[corr_features], diag_kind="kde", markers="+", hue="target")
plt.show()

<a id="3"></a>
<font color='purple'>
## Outlier Detection

In [ ]:
y=data.target
x=data.drop(["target"], axis=1)
columns=x.columns.tolist()

In [ ]:
clf=LocalOutlierFactor()
y_pred=clf.fit_predict(x)
X_score=clf.negative_outlier_factor_
y_pred

-1's are outliers. 1's are inliers.

In [ ]:
outlier_score=pd.DataFrame()
outlier_score["score"]=X_score
outlier_score.head()

In [ ]:
outlier_score

In [ ]:
threshold=-2.5
filtre=outlier_score["score"]< threshold
outlier_index=outlier_score[filtre].index.tolist()

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(x.iloc[outlier_index,0], x.iloc[outlier_index,1], color="blue",s=50 ,label="outliers")
plt.scatter(x.iloc[:,0], x.iloc[:,1], color="k", s=3 , label="Data Points")
radius=(X_score.max()-X_score)/(X_score.max()-X_score.min())
outlier_score["radius"]=radius
plt.scatter(x.iloc[:,0], x.iloc[:,1], s=1000*radius, edgecolors="r", facecolors="none", label="outlier scores")
plt.legend()
plt.show()

Drop outliers:

In [ ]:
x=x.drop(outlier_index)
y=y.drop(outlier_index).values

<a id="4"></a>
<font color='purple'>
## Train Test Split

In [ ]:
test_size=0.3
X_train, X_test, Y_train, Y_test=train_test_split(x,y, test_size=test_size, random_state=42)

<a id="4"></a>
<font color='purple'>
## Standardization

In [ ]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
X_train_df=pd.DataFrame(X_train, columns=columns)
X_train_df["target"]=Y_train

data_melted=pd.melt(X_train_df, id_vars="target", var_name="features", value_name="value")
plt.figure(figsize=(20,10))
sns.boxplot(x="features", y="value", hue="target", data=data_melted)
plt.xticks(rotation=90)
plt.show()

<a id="5"></a>
<font color='purple'>
## KNN

In [ ]:
knn=KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train, Y_train)
y_pred=knn.predict(X_test)
cm= confusion_matrix(Y_test, y_pred)
acc= accuracy_score(Y_test, y_pred)
score= knn.score(X_test,Y_test)
print("accuracy score:",acc)
print("score:",score)
print("confusion matrix:", cm)

<a id="6"></a>
<font color='purple'>
## NCA

In [ ]:
scaler=StandardScaler()
x_scaled=scaler.fit_transform(x)
nca=NeighborhoodComponentsAnalysis(n_components=2, random_state=42)
nca.fit(x_scaled, y)
x_reduced_nca=nca.transform(x_scaled)
nca_data=pd.DataFrame(x_reduced_nca , columns=["p1","p2"])
nca_data["target"]=y


In [ ]:
plt.figure(figsize=(10,5))
sns.scatterplot(x="p1", y="p2" , hue="target", data=nca_data)
plt.title("NCA: p1 vs p2")

In [ ]:
x_train_nca, x_test_nca, y_train_nca, y_test_nca=train_test_split(x_reduced_nca,y, test_size=0.3, random_state=42)
knn=KNeighborsClassifier(n_neighbors=2)
knn.fit(x_train_nca, y_train_nca)
y_pred_NCA=knn.predict(x_test_nca)
cm= confusion_matrix(y_test_nca, y_pred_NCA)
acc= accuracy_score(y_test_nca, y_pred_NCA)
score= knn.score(x_test_nca,y_test_nca)
print("accuracy score:",acc)
print("score:",score)
print("confusion matrix:", cm)